In [32]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import data_utils
import download
from scipy.stats import skew, kurtosis
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import keras
from keras import layers
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from keras import ops

In [33]:
def loadData(path):
    listOfTestFiles = os.listdir(path=path)
    train = []
    train_labels = []
    test = []
    test_labels = []
        
        
    print("Training files = ",listOfTestFiles[1:6])
    #For collecting Training data:
    for file in listOfTestFiles[1:6]:
        with open(path+file,'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            train.append(dict[b'data'])
            train_labels.append(dict[b'labels'])

    print(listOfTestFiles[7])
    #for collecting Testing data
    with open(path+listOfTestFiles[7],'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            test.append(dict[b'data'])
            test_labels.append(dict[b'labels'])

    dictData = {}
    dictData['train_data'] = np.reshape(np.array(train),newshape=(np.array(train).shape[0]*np.array(train).shape[1],np.array(train).shape[2]))
    dictData['train_labels'] = np.reshape(np.array(train_labels),newshape=(np.array(train_labels).shape[0]*np.array(train_labels).shape[1]))
    dictData['test_data'] = np.reshape(np.array(test),newshape=(np.array(test).shape[0]*np.array(test).shape[1],np.array(test).shape[2]))
    dictData['test_labels'] = np.reshape(np.array(test_labels),newshape=(np.array(test_labels).shape[0]*np.array(test_labels).shape[1]))
    return dictData

In [34]:
class kNearestNeighbour(object):
    def __init__(self):
        pass
    
    def train(self, X, Y):
        # Knn will remember all of its training data
        self.Xtr = X
        self.Ytr = Y
        
    def predict(self, X, k):
        # Ensure k is not larger than the training dataset
        if k > self.Xtr.shape[0]:
            raise ValueError(f"k={k} is greater than the number of training samples={self.Xtr.shape[0]}")
        
        # Number of test samples
        test_samples = X.shape[0]
        Ypred = np.zeros(test_samples, dtype=self.Ytr.dtype)
        
        for i in range(test_samples):
            print(f"Test example = {i}", end="\r")
            
            # Compute L1 distances
            dist = np.sum(np.abs(X[i, :] - self.Xtr), axis=1)
            
            # Find the indices of the k smallest distances
            idx = np.argpartition(dist, k)[:k]
            
            # Initialize weights for each label
            label_weights = np.zeros(10, dtype=np.float32)
            
            for x in idx:
                distance = dist[x]
                if distance == 0:
                    weight = 1e6  # Assign a very high weight if distance is 0 to avoid division by zero
                else:
                    weight = 1 / distance
                
                # Accumulate weights for the corresponding label
                label_weights[int(self.Ytr[x])] += weight
            
            # Assign the label with the maximum weighted sum
            Ypred[i] = np.argmax(label_weights)
        
        return Ypred


In [35]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [36]:
# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the data
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # (50000, 32*32*3)
x_test_flat = x_test.reshape(x_test.shape[0], -1)    # (10000, 32*32*3)

# Reshape labels to 1D
y_train = y_train.flatten()
y_test = y_test.flatten()

In [40]:
# Initialize the KNN classifier
knn = kNearestNeighbour()

# Train the classifier
knn.train(x_train_flat, y_train)

# Predict the labels for a subset of test data (e.g., 100 samples)
num_test_samples = 100  # To save time, use a small subset for testing
y_pred = knn.predict(x_test_flat[:num_test_samples], k=20)

# Calculate accuracy
accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.37 99


In [38]:
# Apply LDA
lda = LinearDiscriminantAnalysis(n_components=9)  # CIFAR-10 has 10 classes, so max components = 10 - 1
x_train_lda = lda.fit_transform(x_train_flat, y_train)
x_test_lda = lda.transform(x_test_flat)

In [41]:
# Train the classifier after LDA
knn.train(x_train_lda, y_train)
k_values = [2, 5, 10, 15, 25, 50]  # Test k from 1 to 10

# Store the accuracies for each k
accuracies = []

# Loop through each k value
for k in k_values:
    print(f"Testing for k = {k}")
    
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_lda[:num_test_samples], k=k)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print summary
print("\nSummary of accuracies for different k values:")
for k, acc in zip(k_values, accuracies):
    print(f"k = {k}: Accuracy = {acc * 100:.2f}%")


Testing for k = 1
Accuracy for k = 1: 30.00%
Testing for k = 2
Accuracy for k = 2: 30.00%
Testing for k = 3
Accuracy for k = 3: 34.00%
Testing for k = 4
Accuracy for k = 4: 38.00%
Testing for k = 5
Accuracy for k = 5: 38.00%
Testing for k = 6
Accuracy for k = 6: 35.00%
Testing for k = 7
Accuracy for k = 7: 40.00%
Testing for k = 8
Accuracy for k = 8: 39.00%
Testing for k = 9
Accuracy for k = 9: 40.00%
Testing for k = 10
Accuracy for k = 10: 37.00%

Summary of accuracies for different k values:
k = 1: Accuracy = 30.00%
k = 2: Accuracy = 30.00%
k = 3: Accuracy = 34.00%
k = 4: Accuracy = 38.00%
k = 5: Accuracy = 38.00%
k = 6: Accuracy = 35.00%
k = 7: Accuracy = 40.00%
k = 8: Accuracy = 39.00%
k = 9: Accuracy = 40.00%
k = 10: Accuracy = 37.00%
